<a href="https://colab.research.google.com/github/aimetz/Bears/blob/master/BlankJMetzA_Investigation_1%2C_Question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
N = 10

grid = pd.DataFrame(data={"x":np.repeat(np.arange(1, N+1), N), "y":np.tile(np.arange(1, N+1), N)})

grid["prior"] = (np.exp(-0.3 * (grid["x"] - 3) ** 2 + -0.5 * (grid["y"] - 3) ** 2) + np.exp(-0.4 * (grid["x"] - 7) ** 2 - 0.2 * (grid["y"] - 7) ** 2))
grid["prior"] = grid["prior"]/grid["prior"].sum()
grid.index = grid.index+1
grid

,x,y,prior
1,1,1,0.002162
2,1,2,0.009692
3,1,3,0.015979
4,1,4,0.009692
5,1,5,0.002162
...,...,...,...
96,10,6,0.001187
97,10,7,0.001450
98,10,8,0.001187
99,10,9,0.000651


In [2]:
def organize(df, col):
    map = df.pivot_table(values=col, index="y", columns='x', aggfunc='first')
    map = map.iloc[::-1]
    return map

In [3]:
organize(grid, "prior").style.background_gradient(cmap ='viridis')

x,1,2,3,4,5,6,7,8,9,10
y,,,,,,,,,,
10,0.000000,0.000000,0.000015,0.000240,0.001770,0.005878,0.008769,0.005878,0.001770,0.000240
9,0.000000,0.000001,0.000040,0.000651,0.004813,0.015979,0.023837,0.015979,0.004813,0.000651
8,0.000000,0.000002,0.000072,0.001187,0.008769,0.029115,0.043434,0.029115,0.008769,0.001187
7,0.000005,0.000016,0.000106,0.001463,0.010716,0.035562,0.053051,0.035561,0.010711,0.001450
6,0.000178,0.000439,0.000662,0.001623,0.008947,0.029155,0.043439,0.029115,0.008769,0.001187
5,0.002162,0.005320,0.007219,0.005970,0.006975,0.016461,0.023896,0.015983,0.004813,0.000651
4,0.009692,0.023838,0.032192,0.024077,0.011462,0.008041,0.009034,0.005896,0.001771,0.000240
3,0.015979,0.039301,0.053054,0.039360,0.016415,0.005015,0.002599,0.001479,0.000438,0.000059
2,0.009692,0.023837,0.032178,0.023847,0.009764,0.002402,0.000622,0.000257,0.000073,0.000010


In [4]:
grid.loc[grid["prior"].idxmax()]

x        3.000000
y        3.000000
prior    0.053054
Name: 23, dtype: float64

#1)Square (3, 3) is searched on day 1 and the plane is not found that square. Compute the posterior distribution after day 1, and represent it in a tile plot like the one for the prior. Compare the posterior distribution to the prior; how has it changed? Which location will you search on day 2? Hint: the key is to properly defining the likelihood.

In [5]:
p = grid.loc[grid["prior"].idxmax()]["prior"]
grid["post1"] = grid["prior"] /(1-p*.1)
grid["post1"].loc[grid["prior"].idxmax()] = (.9*p)/((1-p)+p*.9)
organize(grid, "post1").style.background_gradient(cmap ='viridis')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


x,1,2,3,4,5,6,7,8,9,10
y,,,,,,,,,,
10,0.000000,0.000000,0.000015,0.000241,0.001780,0.005910,0.008816,0.005910,0.001780,0.000241
9,0.000000,0.000001,0.000040,0.000655,0.004838,0.016064,0.023964,0.016064,0.004838,0.000655
8,0.000000,0.000002,0.000073,0.001193,0.008816,0.029270,0.043666,0.029270,0.008816,0.001193
7,0.000005,0.000016,0.000107,0.001471,0.010773,0.035752,0.053334,0.035751,0.010768,0.001457
6,0.000178,0.000441,0.000665,0.001632,0.008994,0.029310,0.043671,0.029271,0.008816,0.001193
5,0.002174,0.005348,0.007258,0.006002,0.007012,0.016549,0.024024,0.016068,0.004838,0.000655
4,0.009743,0.023965,0.032363,0.024205,0.011523,0.008084,0.009082,0.005927,0.001781,0.000241
3,0.016064,0.039511,0.048004,0.039570,0.016503,0.005042,0.002613,0.001487,0.000440,0.000059
2,0.009743,0.023964,0.032349,0.023974,0.009816,0.002415,0.000626,0.000259,0.000073,0.000010


In [6]:
grid.loc[grid["post1"].idxmax()]

x        7.000000
y        7.000000
prior    0.053051
post1    0.053334
Name: 67, dtype: float64

Square (3,3)/number 23 went down and the rest went up by a little. I would search (7, 7)/square number 67 on day 2

#2) Use the posterior after day 1 as the prior for day 2. The square that you identified in the previous part is searched on day 2, and the plane is not found in the square. Compute the posterior distribution after day 2, and represent it in a tile plot like the one for the prior. Compare the posterior distribution after day 2 to the one from after day 1; how has it changed? Which location will you search on day 3?

In [7]:
p = grid.loc[grid["post1"].idxmax()]["post1"]
grid["post2"] = grid["post1"] /(1-p*.1)
grid["post2"].loc[grid["post1"].idxmax()] = (.9*p)/((1-p)+p*.9)
organize(grid, "post2").style.background_gradient(cmap ='viridis')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


x,1,2,3,4,5,6,7,8,9,10
y,,,,,,,,,,
10,0.000000,0.000000,0.000015,0.000242,0.001789,0.005941,0.008863,0.005941,0.001789,0.000242
9,0.000000,0.000001,0.000040,0.000658,0.004864,0.016150,0.024093,0.016150,0.004864,0.000658
8,0.000000,0.000002,0.000073,0.001200,0.008863,0.029427,0.043900,0.029427,0.008863,0.001200
7,0.000005,0.000016,0.000107,0.001478,0.010831,0.035944,0.048258,0.035942,0.010826,0.001465
6,0.000179,0.000443,0.000669,0.001641,0.009043,0.029467,0.043905,0.029427,0.008863,0.001200
5,0.002186,0.005377,0.007297,0.006034,0.007050,0.016638,0.024153,0.016154,0.004864,0.000658
4,0.009795,0.024093,0.032537,0.024335,0.011585,0.008127,0.009131,0.005959,0.001790,0.000242
3,0.016150,0.039723,0.048261,0.039782,0.016591,0.005069,0.002627,0.001495,0.000442,0.000060
2,0.009795,0.024093,0.032523,0.024103,0.009868,0.002428,0.000629,0.000260,0.000074,0.000010


In [8]:
grid.loc[grid["post2"].idxmax()]

x        3.000000
y        3.000000
prior    0.053054
post1    0.048004
post2    0.048261
Name: 23, dtype: float64

Square (7,7)/number 67 went down and the rest went up by a little. I would search (3,3)/square number 23 on day 3

#3) Continue in this manner for 10 days of unsuccessful searches. At the end of each day report the square that has the highest posterior probability, and what the probability is. Also provide a plot of the posterior distribution after the 10 days; how does it compare to the prior?

In [9]:
for i in range(3, 11):
  p = grid.loc[grid["post"+str(i-1)].idxmax()]["post"+str(i-1)]
  grid["post"+str(i)] = grid["post"+str(i-1)] /(1-p*.1)
  grid["post"+str(i)].loc[grid["post"+str(i-1)].idxmax()] = (.9*p)/((1-p)+p*.9)
organize(grid, "post10").style.background_gradient(cmap ='viridis')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


x,1,2,3,4,5,6,7,8,9,10
y,,,,,,,,,,
10,0.000000,0.000000,0.000015,0.000251,0.001855,0.006157,0.009185,0.006157,0.001855,0.000251
9,0.000000,0.000001,0.000041,0.000682,0.005041,0.016737,0.024969,0.016737,0.005041,0.000682
8,0.000000,0.000002,0.000076,0.001243,0.009185,0.030497,0.040946,0.030497,0.009185,0.001243
7,0.000006,0.000016,0.000111,0.001532,0.011225,0.037250,0.040510,0.037249,0.011219,0.001518
6,0.000186,0.000459,0.000693,0.001700,0.009371,0.030538,0.040951,0.030497,0.009185,0.001243
5,0.002265,0.005572,0.007562,0.006253,0.007306,0.017242,0.025030,0.016741,0.005041,0.000682
4,0.010151,0.024969,0.033719,0.025220,0.012006,0.008422,0.009463,0.006176,0.001855,0.000251
3,0.016737,0.037050,0.040512,0.037105,0.017194,0.005253,0.002722,0.001549,0.000458,0.000062
2,0.010151,0.024969,0.033705,0.024979,0.010227,0.002516,0.000652,0.000270,0.000076,0.000010


In [10]:
grid.loc[grid["post10"].idxmax()]

x         7.000000
y         6.000000
prior     0.043439
post1     0.043671
post2     0.043905
post3     0.044118
post4     0.044333
post5     0.040077
post6     0.040257
post7     0.040436
post8     0.040616
post9     0.040783
post10    0.040951
Name: 66, dtype: float64

After all 10 days I would search (7,6)

In [11]:
df = pd.DataFrame(columns=["x", "y", "prob"])
best = pd.DataFrame(data={"idx" : grid.idxmax()[2:]})
for col, val in zip(best.index, best.idx):
  df.loc[col] = grid.loc[val][["x","y",col]].rename({col:"prob"})
df["idx"] = best.idx
df

,x,y,prob,idx
prior,3.0,3.0,0.053054,23
post1,7.0,7.0,0.053334,67
post2,3.0,3.0,0.048261,23
post3,7.0,7.0,0.048492,67
post4,7.0,6.0,0.044333,66
post5,7.0,8.0,0.044525,68
post6,3.0,3.0,0.044251,23
post7,7.0,7.0,0.044444,67
post8,4.0,3.0,0.040891,33
post9,2.0,3.0,0.040998,13


#4) Moving the search operation around the ocean is costly. Suppose that you decide to spend the first 4 days searching square (3, 3). If you don’t find the plane after 4 days of searching square (3, 3) what is the posterior distribution? Note: you should do this in a single Bayes table, with the appropriate likelihood. (Yes, you could do it sequentially like in the previous parts, but I want you to do it in a single table.) (Also, yes, we could just adjust p properly to translate from 1-day searches to 4-day searches, but just take the set up as is.)

In [12]:
N = 10

grid = pd.DataFrame(data={"x":np.repeat(np.arange(1, N+1), N), "y":np.tile(np.arange(1, N+1), N)})

grid["prior"] = (np.exp(-0.3 * (grid["x"] - 3) ** 2 + -0.5 * (grid["y"] - 3) ** 2) + np.exp(-0.4 * (grid["x"] - 7) ** 2 - 0.2 * (grid["y"] - 7) ** 2))
grid["prior"] = grid["prior"]/grid["prior"].sum()
grid.index = grid.index+1
grid

,x,y,prior
1,1,1,0.002162
2,1,2,0.009692
3,1,3,0.015979
4,1,4,0.009692
5,1,5,0.002162
...,...,...,...
96,10,6,0.001187
97,10,7,0.001450
98,10,8,0.001187
99,10,9,0.000651


In [13]:
grid.prior.idxmax()

23

In [14]:
grid["post0"] = grid.prior
grid

,x,y,prior,post0
1,1,1,0.002162,0.002162
2,1,2,0.009692,0.009692
3,1,3,0.015979,0.015979
4,1,4,0.009692,0.009692
5,1,5,0.002162,0.002162
...,...,...,...,...
96,10,6,0.001187,0.001187
97,10,7,0.001450,0.001450
98,10,8,0.001187,0.001187
99,10,9,0.000651,0.000651


In [15]:
for i in range(1, 5):
  p = grid.loc[23]["post"+str(i-1)]
  grid["post"+str(i)] = grid["post"+str(i-1)] /(1-p*.1)
  grid["post"+str(i)].loc[23] = (.9*p)/((1-p)+p*.9)
grid.drop(["post0"], axis=1)
organize(grid, "post4").style.background_gradient(cmap ='viridis')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


x,1,2,3,4,5,6,7,8,9,10
y,,,,,,,,,,
10,0.000000,0.000000,0.000015,0.000244,0.001803,0.005987,0.008932,0.005987,0.001803,0.000244
9,0.000000,0.000001,0.000040,0.000663,0.004902,0.016276,0.024280,0.016276,0.004902,0.000663
8,0.000000,0.000002,0.000074,0.001209,0.008932,0.029656,0.044242,0.029656,0.008932,0.001209
7,0.000005,0.000016,0.000108,0.001490,0.010915,0.036223,0.054037,0.036222,0.010910,0.001476
6,0.000181,0.000447,0.000674,0.001654,0.009113,0.029696,0.044247,0.029656,0.008932,0.001209
5,0.002203,0.005419,0.007353,0.006081,0.007105,0.016767,0.024340,0.016280,0.004902,0.000663
4,0.009872,0.024281,0.032790,0.024524,0.011675,0.008190,0.009202,0.006006,0.001804,0.000244
3,0.016276,0.040032,0.035456,0.040092,0.016720,0.005108,0.002647,0.001506,0.000446,0.000060
2,0.009872,0.024280,0.032776,0.024290,0.009945,0.002447,0.000634,0.000262,0.000074,0.000010


#5)Suppose instead of searching based on highest probability, we just search the squares in order of location: (1, 1) then (1, 2) then (1, 3) etc. Suppose that we search every square exactly once and we never find the plane. What will the posterior distribution be after the results of these 100 unsuccessful searcher? Explain without doing any calculations.

In [16]:
N = 10

grid = pd.DataFrame(data={"x":np.repeat(np.arange(1, N+1), N), "y":np.tile(np.arange(1, N+1), N)})

grid["prior"] = (np.exp(-0.3 * (grid["x"] - 3) ** 2 + -0.5 * (grid["y"] - 3) ** 2) + np.exp(-0.4 * (grid["x"] - 7) ** 2 - 0.2 * (grid["y"] - 7) ** 2))
grid["prior"] = grid["prior"]/grid["prior"].sum()
grid.index = grid.index+1
grid

,x,y,prior
1,1,1,0.002162
2,1,2,0.009692
3,1,3,0.015979
4,1,4,0.009692
5,1,5,0.002162
...,...,...,...
96,10,6,0.001187
97,10,7,0.001450
98,10,8,0.001187
99,10,9,0.000651


In [18]:
grid["post0"] = grid.prior
for i in range(1, 101):
  p = grid.loc[i]["post"+str(i-1)]
  grid["post"+str(i)] = grid["post"+str(i-1)] /(1-p*.1)
  grid["post"+str(i)].loc[i] = (.9*p)/((1-p)+p*.9)
grid.drop(["post0"], axis=1)
organize(grid, "post100").style.background_gradient(cmap ='viridis')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


x,1,2,3,4,5,6,7,8,9,10
y,,,,,,,,,,
10,0.000000,0.000000,0.000015,0.000240,0.001770,0.005878,0.008769,0.005878,0.001770,0.000240
9,0.000000,0.000001,0.000040,0.000651,0.004813,0.015979,0.023837,0.015979,0.004813,0.000651
8,0.000000,0.000002,0.000072,0.001187,0.008769,0.029115,0.043434,0.029115,0.008769,0.001187
7,0.000005,0.000016,0.000106,0.001463,0.010716,0.035562,0.053051,0.035561,0.010711,0.001450
6,0.000178,0.000439,0.000662,0.001623,0.008947,0.029155,0.043439,0.029115,0.008769,0.001187
5,0.002162,0.005320,0.007219,0.005970,0.006975,0.016461,0.023896,0.015983,0.004813,0.000651
4,0.009692,0.023838,0.032192,0.024077,0.011462,0.008041,0.009034,0.005896,0.001771,0.000240
3,0.015979,0.039301,0.053054,0.039360,0.016415,0.005015,0.002599,0.001479,0.000438,0.000059
2,0.009692,0.023837,0.032178,0.023847,0.009764,0.002402,0.000622,0.000257,0.000073,0.000010


In [23]:
np.sqrt(((grid.prior-grid.post100)**2).sum())

1.2566087518846303e-16

the last posterior is almost identical to the prior